In [1]:
import json
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
fold = 3
ann = json.load(open(f'/kaggle/input/vizwiz-val/VizWiz-val/Annotation.json'))
ann = ann[1500 * (fold-1): min(1500 * fold, len(ann))]
len(ann)

1319

In [4]:
data_ann, images = [], []
for dp in tqdm(ann):
    image_dir = f"/kaggle/input/vizwiz-val/VizWiz-val/val/val/{dp['image']}"
    data_ann.append({
        'question': dp['question'],
        'answers': dp['answers']
    })
    images.append(Image.open(image_dir).convert('RGB'))

100%|██████████| 1319/1319 [00:33<00:00, 39.47it/s]


In [5]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [6]:
model.eval()
with torch.no_grad():
    for i in tqdm(range(len(data_ann))):
        question = data_ann[i]['question']
        raw_image = images[i]
        inputs = processor(raw_image, question, return_tensors="pt").to(device)
        out = model.generate(**inputs)
        answer = processor.decode(out[0], skip_special_tokens=True)
        data_ann[i]['prediction'] = answer

100%|██████████| 1319/1319 [02:47<00:00,  7.85it/s]


In [7]:
json.dump(data_ann, open(f'BLIP_VizWiz_Val_F{fold}.json', 'w'), indent=4)